In [3]:
##### DEMOS BELOW ###

In [2]:
from colormath.color_objects import LabColor, XYZColor, sRGBColor, HSLColor, AdobeRGBColor
from colormath.color_conversions import convert_color
from colr import Colr as C
from copy import copy, deepcopy

import random as r
import numpy as np
import itertools
import math

from scipy import linalg
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.mixture import GaussianMixture
from matplotlib.patches import Ellipse

class color_library:
    """ Class that represents a color library. The color palette has access to this. """

    def rgb_to_hsl(self, a, b, c):
        rgb = sRGBColor(a, b, c, is_upscaled=True)
        hsl = convert_color(rgb, HSLColor)
        return hsl.get_value_tuple()

    def hsl_to_rgb(self, a, b, c):
        hsl = HSLColor(a, b, c)
        rgb = convert_color(hsl, sRGBColor).get_value_tuple()
        if self.is_valid_rgb(rgb):
            return [rgb[0]*255, rgb[1]*255, rgb[2]*255]
#         return self.correct_rgb(rgb)
        return self.correct_rgb([rgb[0]*255, rgb[1]*255, rgb[2]*255])
        
    def is_valid_rgb(self, color):
        for param in color:
            if param < 0 or param > 255:
                return False;
        return True;
    
    def correct_rgb(self, color):
        color = np.asarray(color)
        for i in range(len(color)):
            if color[i] < 0:
                color[i] = 0
            if color[i] > 255:
                color[i] = 255
        return color
    
    def arr_to_int(self, arr):
        for r in range(len(arr)):
            arr[r] = int(arr[r])
        return arr

    # rgb inputs
    def print_combo(self, fg, bg):
        for i in range(0,3):
            if fg[i] > 255:
                fg[i] = 255
            if bg[i] > 255:
                bg[i]= 255
        print(C().b_rgb(bg[0], bg[1], bg[2]) .rgb(fg[0], fg[1], fg[2], 'Lorem ipsum.'))

    def bound(self, min_val, max_val, val):
        new_val = val
        if (val > max_val):
             new_val = max_val
        elif (val < min_val):
            new_val = min_val
        return new_val
    
    def rgb_to_hex(self, r,g,b):
        return '#%02x%02x%02x' % (int(r), int(g), int(b))
    
    def color_descriptor(self, hue, saturation, lightness):
#         if saturation < 10:
#             return "grey"
        if lightness == 0:
            return "black"
        elif lightness == 100:
            return "white"
        elif hue <= 10 or hue >= 350:
            return "red"
        elif hue < 40:
            return "orange"
        elif hue < 60:
            return "yellow"
        elif hue < 160:
            return "green"
        elif hue < 250:
            return "blue"
        elif hue < 290:
            return "purple"
        elif hue < 350:
            return "pink"
        


In [14]:
class palette_generator:
    """ Class that represents the palette generator model """

    def __init__(self):
        self.color_library = color_library()
        self.color_gmm = GaussianMixture(n_components=3)
        self.hue_gmm = GaussianMixture(n_components=2)
        self.num_steps = 8
        self.num_palettes = 0
        self.randomness = .3 # given the color passed into the stepping wheel, we should diverge from
                             # that color by a certain amount in the beginning to get diverse outputs
                             # this number should lower as we get more samples
                             # 0: never diverges
                             # 1: always change the color
        self.hue_shift_max = 20
        # an array of hue shift index, which measures how much hue shift is desired (*self.hue_shift_max )
        # index = 0: never shifts (0)
        # index = 1: always shifts (20)
        
    # Helper method for testing
    # input: array of hsl samples directly taken from the gmm and converts them
    # into rgb and prints them!
    def print_samples(self, samples):
        print("----gmm samples----")
        for color in samples:
            rgb = self.color_library.hsl_to_rgb(color[0], color[1], color[2])
            self.color_library.print_combo(rgb,rgb)
            
    # method that takes in rgb colors and palettes that the user likes, and updates
    # hyperparemters such as randomness, num_inputs, and hue preference;
    # this should be called before the stepping wheel, in generate_palettes
    # a palette is a tuple of rgb values like [[r, g, b], [r, g, b]]
    # output: a list of hsl liked colors to be passed into the gmm
    def process_input(self, samples, palettes, if_print):
        hsl_input = []
        print("given samples:")
        for color in samples:
            if if_print: 
                self.color_library.print_combo(color, color)
            hsl_input.append(self.color_library.rgb_to_hsl(color[0], color[1], color[2]))
        print("---")
        # randomness: decreases randomness for every color we like
        self.num_palettes = len(palettes)
        num_inputs = len(samples) + 2 * len(palettes) # assuming each palette has 2 colors
        self.randomness = 1/(0.04 * num_inputs + 1) # over 25 input colors: < 0.5
        
        # hue shift + hue_gmm
        print("given palettes:")
        hue_shifts = []
        for palette in palettes:
            if if_print:
                self.color_library.print_combo(palette[0], palette[1])
                self.color_library.print_combo(palette[1], palette[0])
            color1 = self.color_library.rgb_to_hsl(palette[0][0], palette[0][1], palette[0][2])
            color2 = self.color_library.rgb_to_hsl(palette[1][0], palette[1][1], palette[1][2])
            hsl_input.append(color1)
            hsl_input.append(color2)
            hue_diff = math.radians(abs(color1[0] - color2[0]))
#             hue_shift = (0.5 * math.sin(hue_diff - 0.5 * math.pi)) + 0.5 # 0/360 -> 0 / 180 -> 1
            hue_shift = 1 - abs(1/math.pi*(hue_diff - math.pi))
            hue_shifts.append(hue_shift)
        print("---")
        if self.num_palettes >= 2:    
            hue_shifts = np.reshape(hue_shifts, (-1, 1))
            self.hue_gmm.fit(hue_shifts)
        return hsl_input
    
    def generate_hue_shift(self):
        if self.num_palettes < 2:
            hue_shift = 0.5
        else:
            hs = np.clip(self.hue_gmm.sample(1)[0][0][0], 0, 1)
            if hs > 0.4 and hs < 0.6: # medium shift
                if r.random() < 0.3:
                    hue_shift = abs(r.uniform(0, 0.4) * self.hue_shift_max)
                elif r.random() > 0.7:
                    hue_shift = abs(r.uniform(0.6, 1) * self.hue_shift_max)
                else: # 40% of the time
                    hue_shift = abs(r.uniform(0.4, 0.6) * self.hue_shift_max)
            else:
                if r.random() < hs:
                    hue_shift = abs(r.uniform(hs, 1) * self.hue_shift_max)
                else:
                    hue_shift = abs(r.uniform(0, hs) * self.hue_shift_max)
        return hue_shift
        # 0.1: 10% (0.1, 1.0) | 90% (0, 0.1)
    
    # enabling other hues to be generated
    def randomnize_given_color(self, color):
        new_color = deepcopy(color)
        if r.random() < self.randomness:
            new_hue = np.mod(color[0] + r.uniform(30, 100), 360)
            color[0] = new_hue
            new_saturation = self.saturation_clip(color[1] + r.uniform(0, 0.2))
            color[1] = new_saturation
        return color
    
    def saturation_clip(self, value):
        value = abs(value)
        direction = (-1)**int(value)
        return np.mod((1 + direction*np.mod(value, 1)), 1)
    
    def stepping_wheel(self, color):
        self.num_steps= round(r.uniform(6, 8))
        
        rgb = self.color_library.hsl_to_rgb(color[0], color[1], color[2])
        # randomnize colors based on self.randomness to get more diverse outputs
        color = self.randomnize_given_color(color)
        rgb = self.color_library.hsl_to_rgb(color[0], color[1], color[2])

        # center_i: middle of the palette
        center_i = round(self.num_steps/2 + 0.1)-1
            
        # hue: linear increase
        palette_h = np.zeros(self.num_steps)
        hue_shift = self.generate_hue_shift()
        for i in range(0, self.num_steps):
            value = color[0] + (i - center_i) * hue_shift
            palette_h[i] = np.mod(value, 360)
        
        # saturation: proportional decrease
        palette_s = np.zeros(self.num_steps)         
        saturation_shift = 0.2
        palette_s[center_i] = np.clip(color[1], 0, 1)
        sign = 1
        for i in range(center_i - 1, -1, -1): # left
            last = palette_s[i + 1] # saturation of last element
            prop = 1.0/(1 + np.exp(-abs(i - center_i)/5.0))  # sigmoid
            value = sign*last - saturation_shift * prop
            sign = value / abs(value)
            palette_s[i] = self.saturation_clip(value)
        sign = 1
        for i in range(center_i + 1, self.num_steps, 1): # right   
            last = palette_s[i - 1] # saturation of last element
            prop = 1.0/(1 + np.exp(-abs(i - center_i)/5.0)) 
            value = sign*last - saturation_shift * prop
            sign = value / abs(value)
            palette_s[i] = self.saturation_clip(value)
        palette_s = np.clip(palette_s, 0.01, 0.99)
        
        # brightness: a*log(x)+1, a in [0.3, 1] (increasing)
        palette_b = np.zeros(self.num_steps)
        # determines the placement of the sampled color in the palette
        center_i = round(self.num_steps/(1.0 + np.exp(3-5*color[2]))) # modified sigmoid
        center_i = np.clip(int(center_i), 0, self.num_steps-1)
        palette_b[center_i] = np.clip(color[2], 0.01, 0.95) # given middle color
        delta_x = 0.05
        # setting a in the a*log(x) + 1
        if (color[2] > 0.49 and color[2] < 0.51): # heuristics for setting the rate of change of brightness
            a = 1.585
        else:
            a = abs(0.1/(color[2]-0.5))**0.2 
        for i in range(center_i - 1, -1, -1): # left
            last = palette_b[i + 1] # brightness of last element
            palette_b[i] = np.clip((last - delta_x * (a/last)**0.5), 0.01, 0.95) # derivative of a*log(x)+1  
        for i in range(center_i + 1, self.num_steps, 1): # right   
            last = palette_b[i - 1] # brightness of last element
            palette_b[i] = np.clip((last + delta_x * (a/last)**0.5), 0.01, 0.95) # derivative of a*log(x)+1
        palette_b = np.clip(palette_b, 0.01, 0.95)
        
        # combining h, s, b
        palette = []
        for i in range(0, self.num_steps):
            c = [palette_h[i], palette_s[i], palette_b[i]]
            palette.append(c) 
            rgb = self.color_library.hsl_to_rgb(c[0], c[1], c[2])
            self.color_library.print_combo(rgb, rgb)

        return palette
    
    
    # Given a list of inputs [colors] that are in hsl form [[h,s,l], [h,s,l], [h,s,l] ...]
    # Outputs colors in HSL: [[h,s,l], [h,s,l], [h,s,l]...]
    def sample_gmm(self, hsl_input, num_samples):
        hsl_input = np.reshape(hsl_input, (-1, 3)) # 3 columns
        self.color_gmm.fit(hsl_input)
        return self.color_gmm.sample(num_samples)[0]
    
    
    # takes in liked colors from quiz and liked palettes?
    # return palettes in form [[[h,s,l], [h,s,l]...], [[h,s,l], [h,s,l] ...]].
    def generate_palettes(self, samples, palettes, num_palettes, if_print):
        hsl_input = self.process_input(samples, palettes, if_print)
        gmm_samples = self.sample_gmm(hsl_input, num_palettes)
        palettes = []
        for color in gmm_samples: # make a palette for each sample from the gmm
            p = self.stepping_wheel(color)
            palettes.append(p)
            rgb1 = self.color_library.hsl_to_rgb(p[0][0], p[0][1], p[0][2])
            rgb2 = self.color_library.hsl_to_rgb(p[self.num_steps - 1][0], p[self.num_steps - 1][1], p[self.num_steps - 1][2])
            self.color_library.print_combo(rgb1, rgb2)
            self.color_library.print_combo(rgb2, rgb1)
            print("")
        return palettes
    
    # helper function that translates generated palettes into a json file. (outputs of generate_palettes)
    # inputs: palettes in form [ [ color, .., color], [color, ..., color] ]  where each color is hsl
    # output: a json file in format:
    # [{id: color1{label, hex, rgb}, {color2:label, hex, rgb}]
    def output_to_json(self, palettes):
        outputs = {}
        
        for palette in palettes:
            c_counter = 1
            color_id = []
            p = {}
            for color in palette:
                c = {}
                rgb = self.color_library.hsl_to_rgb(color[0], color[1], color[2])
                c["rgb"] = self.color_library.arr_to_int(rgb)
                c["hex"] = self.color_library.rgb_to_hex(rgb[0], rgb[1], rgb[2])
                c["label"] = [self.color_library.color_descriptor(color[0], color[1], color[2])]
                color_string = "color" + str(c_counter)
                p[color_string] = c
                c_counter +=1
                color_id.append(c["hex"])
            
            outputs[hash(tuple(color_id))] = p # note: we want to order the palettes from light to dark before hashing!
        
        return outputs

In [19]:
# process_input: randomness + hue_shift -- Anna 
class palette_generator:
    """ Class that represents the palette generator model """

    def __init__(self):
        self.color_library = color_library()
        self.color_gmm = GaussianMixture(n_components=3)
        self.hue_gmm = GaussianMixture(n_components=2)
        self.num_steps = 8
        self.num_palettes = 0
        self.randomness = .3 # given the color passed into the stepping wheel, we should diverge from
                             # that color by a certain amount in the beginning to get diverse outputs
                             # this number should lower as we get more samples
                             # 0: never diverges
                             # 1: always change the color
        self.hue_shift_max = 20
        # an array of hue shift index, which measures how much hue shift is desired (*self.hue_shift_max )
        # index = 0: never shifts (0)
        # index = 1: always shifts (20)
        
    # Helper method for testing
    # input: array of hsl samples directly taken from the gmm and converts them
    # into rgb and prints them!
    def print_samples(self, samples):
        print("----gmm samples----")
        for color in samples:
            rgb = self.color_library.hsl_to_rgb(color[0], color[1], color[2])
            self.color_library.print_combo(rgb,rgb)
            
    # method that takes in rgb colors and palettes that the user likes, and updates
    # hyperparemters such as randomness, num_inputs, and hue preference;
    # this should be called before the stepping wheel, in generate_palettes
    # a palette is a tuple of rgb values like [[r, g, b], [r, g, b]]
    # output: a list of hsl liked colors to be passed into the gmm
    def process_input(self, samples, palettes, if_print):
        hsl_input = []
        print("given samples:")
        for color in samples:
            if if_print: 
                self.color_library.print_combo(color, color)
            hsl_input.append(self.color_library.rgb_to_hsl(color[0], color[1], color[2]))
        print("---")
        # randomness: decreases randomness for every color we like
        self.num_palettes = len(palettes)
        num_inputs = len(samples) + 2 * len(palettes) # assuming each palette has 2 colors
        self.randomness = 1/(0.04 * num_inputs + 1) # over 25 input colors: < 0.5
        print("randomness: ", self.randomness)
        
        # hue shift (-|x|) + hue_gmm
        print("given palettes:")
        hue_shifts = []
        for palette in palettes:
            if if_print:
                self.color_library.print_combo(palette[0], palette[1])
                self.color_library.print_combo(palette[1], palette[0])
            color1 = self.color_library.rgb_to_hsl(palette[0][0], palette[0][1], palette[0][2])
            color2 = self.color_library.rgb_to_hsl(palette[1][0], palette[1][1], palette[1][2])
            hsl_input.append(color1)
            hsl_input.append(color2)
            print("hue difference: ", abs(color1[0] - color2[0]))
            hue_diff = math.radians(abs(color1[0] - color2[0]))
#             hue_shift = (0.5 * math.sin(hue_diff - 0.5 * math.pi)) + 0.5 # 0/360 -> 0 / 180 -> 1
            hue_shift = 1 - abs(1/math.pi*(hue_diff - math.pi))
            print("hue shift: ", hue_shift)
            hue_shifts.append(hue_shift)
        print("---")
        if self.num_palettes >= 2:    
            hue_shifts = np.reshape(hue_shifts, (-1, 1))
            self.hue_gmm.fit(hue_shifts)
            print("hue_gmm means: ", self.hue_gmm.means_)
        return hsl_input
    
    def generate_hue_shift(self):
        if self.num_palettes < 2:
            hue_shift = 0.5
        else:
            hs = np.clip(self.hue_gmm.sample(1)[0][0][0], 0, 1)
            print("sampled hue shift index", hs)
            if hs > 0.4 and hs < 0.6: # medium shift
                if r.random() < 0.3:
                    hue_shift = abs(r.uniform(0, 0.4) * self.hue_shift_max)
                elif r.random() > 0.7:
                    hue_shift = abs(r.uniform(0.6, 1) * self.hue_shift_max)
                else: # 40% of the time
                    hue_shift = abs(r.uniform(0.4, 0.6) * self.hue_shift_max)
            else:
                if r.random() < hs:
                    hue_shift = abs(r.uniform(hs, 1) * self.hue_shift_max)
                else:
                    hue_shift = abs(r.uniform(0, hs) * self.hue_shift_max)
        print("generated hue shift", hue_shift)
        return hue_shift
        # 0.1: 10% (0.1, 1.0) | 90% (0, 0.1)
    
    # enabling other hues to be generated
    def randomnize_given_color(self, color):
        new_color = deepcopy(color)
        if r.random() < self.randomness:
            print("sampled color changed from ", color)
            c1 = self.color_library.hsl_to_rgb(color[0], color[1], color[2])
            self.color_library.print_combo(c1, c1)
            new_hue = np.mod(color[0] + r.uniform(30, 100), 360)
            color[0] = new_hue
            new_saturation = self.saturation_clip(color[1] + r.uniform(0, 0.2))
            color[1] = new_saturation
            print("to", color)
            c2 =  self.color_library.hsl_to_rgb(color[0], color[1], color[2])
            self.color_library.print_combo(c2, c2)
        return color
    
    def saturation_clip(self, value):
        value = abs(value)
        direction = (-1)**int(value)
        return np.mod((1 + direction*np.mod(value, 1)), 1)
    
    def stepping_wheel(self, color):
        self.num_steps= round(r.uniform(6, 8))
        
        rgb = self.color_library.hsl_to_rgb(color[0], color[1], color[2])
        # randomnize colors based on self.randomness to get more diverse outputs
        color = self.randomnize_given_color(color)
        rgb = self.color_library.hsl_to_rgb(color[0], color[1], color[2])

        # center_i: middle of the palette
        center_i = round(self.num_steps/2 + 0.1)-1
            
        # hue: linear increase
        palette_h = np.zeros(self.num_steps)
        hue_shift = self.generate_hue_shift()
        for i in range(0, self.num_steps):
            value = color[0] + (i - center_i) * hue_shift
            palette_h[i] = np.mod(value, 360)
        
        # saturation: proportional decrease
        palette_s = np.zeros(self.num_steps)         
        saturation_shift = 0.2
        palette_s[center_i] = np.clip(color[1], 0, 1)
        sign = 1
        for i in range(center_i - 1, -1, -1): # left
            last = palette_s[i + 1] # saturation of last element
            prop = 1.0/(1 + np.exp(-abs(i - center_i)/5.0))  # sigmoid
            value = sign*last - saturation_shift * prop
            sign = value / abs(value)
            palette_s[i] = self.saturation_clip(value)
        sign = 1
        for i in range(center_i + 1, self.num_steps, 1): # right   
            last = palette_s[i - 1] # saturation of last element
            prop = 1.0/(1 + np.exp(-abs(i - center_i)/5.0)) 
            value = sign*last - saturation_shift * prop
            sign = value / abs(value)
            palette_s[i] = self.saturation_clip(value)
        palette_s = np.clip(palette_s, 0.01, 0.99)
        
        # brightness: a*log(x)+1, a in [0.3, 1] (increasing)
        palette_b = np.zeros(self.num_steps)
        # determines the placement of the sampled color in the palette
        center_i = round(self.num_steps/(1.0 + np.exp(3-5*color[2]))) # modified sigmoid
        center_i = np.clip(int(center_i), 0, self.num_steps-1)
        palette_b[center_i] = np.clip(color[2], 0.01, 0.95) # given middle color
        delta_x = 0.05
        # setting a in the a*log(x) + 1
        if (color[2] > 0.49 and color[2] < 0.51): # heuristics for setting the rate of change of brightness
            a = 1.585
        else:
            a = abs(0.1/(color[2]-0.5))**0.2 
        for i in range(center_i - 1, -1, -1): # left
            last = palette_b[i + 1] # brightness of last element
            palette_b[i] = np.clip((last - delta_x * (a/last)**0.5), 0.01, 0.95) # derivative of a*log(x)+1  
        for i in range(center_i + 1, self.num_steps, 1): # right   
            last = palette_b[i - 1] # brightness of last element
            palette_b[i] = np.clip((last + delta_x * (a/last)**0.5), 0.01, 0.95) # derivative of a*log(x)+1
        palette_b = np.clip(palette_b, 0.01, 0.95)
        
        # combining h, s, b
        palette = []
        for i in range(0, self.num_steps):
            c = [palette_h[i], palette_s[i], palette_b[i]]
            palette.append(c) 
            rgb = self.color_library.hsl_to_rgb(c[0], c[1], c[2])
            self.color_library.print_combo(rgb, rgb)

        return palette
    
    
    # Given a list of inputs [colors] that are in hsl form [[h,s,l], [h,s,l], [h,s,l] ...]
    # Outputs colors in HSL: [[h,s,l], [h,s,l], [h,s,l]...]
    def sample_gmm(self, hsl_input, num_samples):
        hsl_input = np.reshape(hsl_input, (-1, 3)) # 3 columns
        self.color_gmm.fit(hsl_input)
        return self.color_gmm.sample(num_samples)[0]
    
    
    # takes in liked colors from quiz and liked palettes?
    # return palettes in form [[[h,s,l], [h,s,l]...], [[h,s,l], [h,s,l] ...]].
    def generate_palettes(self, samples, palettes, num_palettes, if_print):
        hsl_input = self.process_input(samples, palettes, if_print)
        gmm_samples = self.sample_gmm(hsl_input, num_palettes)
        palettes = []
        for color in gmm_samples: # make a palette for each sample from the gmm
            p = self.stepping_wheel(color)
            palettes.append(p)
            rgb1 = self.color_library.hsl_to_rgb(p[0][0], p[0][1], p[0][2])
            rgb2 = self.color_library.hsl_to_rgb(p[self.num_steps - 1][0], p[self.num_steps - 1][1], p[self.num_steps - 1][2])
            self.color_library.print_combo(rgb1, rgb2)
            self.color_library.print_combo(rgb2, rgb1)
            print("")
        return palettes
    
    # helper function that translates generated palettes into a json file. (outputs of generate_palettes)
    # inputs: palettes in form [ [ color, .., color], [color, ..., color] ]  where each color is hsl
    # output: a json file in format:
    # [{id: color1{label, hex, rgb}, {color2:label, hex, rgb}]
    def output_to_json(self, palettes):
        outputs = {}
        
        for palette in palettes:
            c_counter = 1
            color_id = []
            p = {}
            for color in palette:
                c = {}
                rgb = self.color_library.hsl_to_rgb(color[0], color[1], color[2])
                c["rgb"] = self.color_library.arr_to_int(rgb)
                c["hex"] = self.color_library.rgb_to_hex(rgb[0], rgb[1], rgb[2])
                c["label"] = [self.color_library.color_descriptor(color[0], color[1], color[2])]
                color_string = "color" + str(c_counter)
                p[color_string] = c
                c_counter +=1
                color_id.append(c["hex"])
            
            outputs[hash(tuple(color_id))] = p # note: we want to order the palettes from light to dark before hashing!
        
        return outputs




small_wack = [ [246,71,64], [248,221,164], [191, 219, 247], [60, 187, 177], [87, 226, 229], [241, 113, 5], [106, 16, 242]]
dark = [ [92, 1, 32] ,[72, 4, 4] ,[77, 10, 24] ,[98, 81, 25] ,[1, 55, 26] ,[1, 54, 28] ,[1, 29, 19] ,[1, 39, 49] ,[2, 45, 21] ,[2, 64, 44] ,[9, 35, 15] ,[4, 19, 34] ,[41, 12, 94] ,[38, 3, 104] ,[0, 7, 65] ,[0, 49, 83] ,[1, 13, 26] ,[1, 22, 53] ,[19, 38, 77] ,[22, 42, 64] ,[27, 2, 69] ,[27, 27, 27] ,[13, 3, 50] ,[65, 74, 76] ]
autumn = [ [225, 104, 101] ,[226, 114, 91] ,[222, 99, 96] ,[208, 95, 4] ,[167, 37, 37] ,[179, 45, 41] ,[180, 51, 50] ,[141, 63, 63] ,[255, 231, 114] ,[255, 239, 161] ,[223, 190, 111] ,[225, 188, 100] ,[96, 73, 19] ,[195, 153, 83] ,[202, 187, 72] ,[207, 181, 59] ,[185, 141, 40] ,[186, 127, 3] ,[220, 178, 12] ,[58, 106, 71] ,[57, 100, 19] ,[95, 167, 119] ,[88, 113, 86] ,[93, 94, 55] ,[168, 152, 155] ,[168, 101, 21] ,[171, 145, 122] ,[173, 120, 27] ,[169, 164, 145] ,[175, 89, 62] ,[136, 83, 66] ,[134, 148, 159] ,[138, 51, 36] ,[135, 124, 123] ,[245, 237, 239] ,[255, 228, 205] ,[77, 61, 20] ]

red_palettes =  [[ [255, 45, 80] ,[50, 1, 1] ] , [[255, 33, 72] ,[255, 68, 14] ], [[68, 1, 45] ,[225, 104, 101]]]

very_different = [
[[85, 61, 54],[133, 120, 133]],
[[239, 48, 84], [40 , 80, 46]],
[[15, 163, 177], [237, 22, 164]],
[ [10, 1, 79], [250, 232, 236]],
[[14, 121, 178],[243, 146,55]]]

mixed_palettes = [ [ [85, 61, 54],[133, 120, 133]],
[[239, 48, 84], [40 , 80, 46]],
[ [15, 163, 177], [237, 22, 164]],
[  [10, 1, 79], [250, 232, 236]],
[[ 14, 121, 178],[243, 146,55]],
[[4, 42,43],[84, 242, 242]],
[[58, 183, 149],[237, 234, 208]],
[[20, 13, 79],[78, 166, 153]],
[[110, 164, 191],[236, 254, 232]],
[[9, 21, 64],[171, 210, 250]]]

generator = palette_generator()
palettes = generator.generate_palettes(small_wack, very_different, 10, True)
# palettes = generator.generate_palettes(autumn, red_palettes, 10, True)

given samples:
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
---
randomness:  0.5952380952380952
given palettes:
Lorem ipsum.
Lorem ipsum.
hue difference:  286.4516129032258
hue shift:  0.4086021505376345
Lorem ipsum.
Lorem ipsum.
hue difference:  219.69109947643977
hue shift:  0.7794938917975568
Lorem ipsum.
Lorem ipsum.
hue difference:  135.18690783807065
hue shift:  0.7510383768781703
Lorem ipsum.
Lorem ipsum.
hue difference:  99.7435897435897
hue shift:  0.5541310541310539
Lorem ipsum.
Lorem ipsum.
hue difference:  171.81110534509602
hue shift:  0.9545061408060891
---
hue_gmm means:  [[0.82612486]
 [0.48066259]]
sampled color changed from  [314.34243376   0.5640674    0.52414854]
Lorem ipsum.
to [352.88996758   0.69882994   0.52414854]
Lorem ipsum.
sampled hue shift index 0.5168774228149582
generated hue shift 9.533675115877049
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.

sample

In [20]:
# sample_gmm: Sampling colors to generate palettes (color_gmm) -- Katherine

def gmm_demo(colors, input_name):
    print("inputs: " + input_name)
    color_lib = color_library()
    for c in colors:
        color_lib.print_combo(c,c)
    hsl = []
    
    for c in range(len(colors)):
        hsl.append(color_lib.rgb_to_hsl(colors[c][0], colors[c][1], colors[c][2]))

    generator = palette_generator()
    colors = generator.sample_gmm(hsl, 10)
    print(input_name + " gmm outputs")

    for c in colors:
        rgb = color_lib.hsl_to_rgb(c[0], c[1], c[2])
        color_lib.print_combo(rgb,rgb)
        
dark = [ [92, 1, 32] ,[1, 39, 49] ,[2, 45, 21] ,[2, 64, 44] ,[9, 35, 15] ,[4, 19, 34] ,[41, 12, 94] ,[38, 3, 104] ,[0, 7, 65] ,[0, 49, 83] ,[1, 13, 26] ,[1, 22, 53] ,[19, 38, 77] ,[22, 42, 64] ,[27, 2, 69] ,[27, 27, 27] ,[13, 3, 50] ,[65, 74, 76] ]
underwater = [ [244, 196, 48] ,[240, 213, 45] ,[236, 242, 69] ,[225, 104, 101] ,[222, 99, 96] ,[226, 114, 91] ,[27, 101, 157] ,[25, 89, 168] ,[24, 167, 181] ,[0, 255, 255] ,[28, 169, 201] ,[0, 51, 153] ,[0, 86, 167] ,[0, 35, 135] ,[31, 117, 254] ,[25, 116, 210] ,[0, 47, 167] ,[37, 150, 209] ,[32, 46, 84] ,[37, 31, 79] ,[65, 105, 225] ,[59, 145, 180] ]

gmm_demo(dark, "dark n scary")

gmm_demo(underwater, "underwater")

inputs: dark n scary
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
dark n scary gmm outputs
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
inputs: underwater
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
underwater gmm outputs
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.


In [21]:
# stepping_wheel: How to generate palette (hue, saturation, lightness, steps) -- Anna
class palette_generator:
    """ Class that represents the palette generator model """

    def __init__(self):
        self.color_library = color_library()
        self.color_gmm = GaussianMixture(n_components=3)
        self.hue_gmm = GaussianMixture(n_components=2)
        self.num_steps = 8
        self.num_palettes = 0
        self.randomness = .3 # given the color passed into the stepping wheel, we should diverge from
                             # that color by a certain amount in the beginning to get diverse outputs
                             # this number should lower as we get more samples
                             # 0: never diverges
                             # 1: always change the color
        self.hue_shift_max = 20
        # an array of hue shift index, which measures how much hue shift is desired (*self.hue_shift_max )
        # index = 0: never shifts (0)
        # index = 1: always shifts (20)
        
    # Helper method for testing
    # input: array of hsl samples directly taken from the gmm and converts them
    # into rgb and prints them!
    def print_samples(self, samples):
        print("----gmm samples----")
        for color in samples:
            rgb = self.color_library.hsl_to_rgb(color[0], color[1], color[2])
            self.color_library.print_combo(rgb,rgb)
            
    # method that takes in rgb colors and palettes that the user likes, and updates
    # hyperparemters such as randomness, num_inputs, and hue preference;
    # this should be called before the stepping wheel, in generate_palettes
    # a palette is a tuple of rgb values like [[r, g, b], [r, g, b]]
    # output: a list of hsl liked colors to be passed into the gmm
    def process_input(self, samples, palettes, if_print):
        hsl_input = []
        print("given samples:")
        for color in samples:
            if if_print: 
                self.color_library.print_combo(color, color)
            hsl_input.append(self.color_library.rgb_to_hsl(color[0], color[1], color[2]))
        print("---")
        # randomness: decreases randomness for every color we like
        self.num_palettes = len(palettes)
        num_inputs = len(samples) + 2 * len(palettes) # assuming each palette has 2 colors
        self.randomness = 1/(0.04 * num_inputs + 1) # over 25 input colors: < 0.5
        
        # hue shift + hue_gmm
        print("given palettes:")
        hue_shifts = []
        for palette in palettes:
            if if_print:
                self.color_library.print_combo(palette[0], palette[1])
                self.color_library.print_combo(palette[1], palette[0])
            color1 = self.color_library.rgb_to_hsl(palette[0][0], palette[0][1], palette[0][2])
            color2 = self.color_library.rgb_to_hsl(palette[1][0], palette[1][1], palette[1][2])
            hsl_input.append(color1)
            hsl_input.append(color2)
            hue_diff = math.radians(abs(color1[0] - color2[0]))
#             hue_shift = (0.5 * math.sin(hue_diff - 0.5 * math.pi)) + 0.5 # 0/360 -> 0 / 180 -> 1
            hue_shift = 1 - abs(1/math.pi*(hue_diff - math.pi))
            hue_shifts.append(hue_shift)
        print("---")
        if self.num_palettes >= 2:    
            hue_shifts = np.reshape(hue_shifts, (-1, 1))
            self.hue_gmm.fit(hue_shifts)
        return hsl_input
    
    def generate_hue_shift(self):
        if self.num_palettes < 2:
            hue_shift = 0.5
        else:
            hs = np.clip(self.hue_gmm.sample(1)[0][0][0], 0, 1)
            if hs > 0.4 and hs < 0.6: # medium shift
                if r.random() < 0.3:
                    hue_shift = abs(r.uniform(0, 0.4) * self.hue_shift_max)
                elif r.random() > 0.7:
                    hue_shift = abs(r.uniform(0.6, 1) * self.hue_shift_max)
                else: # 40% of the time
                    hue_shift = abs(r.uniform(0.4, 0.6) * self.hue_shift_max)
            else:
                if r.random() < hs:
                    hue_shift = abs(r.uniform(hs, 1) * self.hue_shift_max)
                else:
                    hue_shift = abs(r.uniform(0, hs) * self.hue_shift_max)
        return hue_shift
        # 0.1: 10% (0.1, 1.0) | 90% (0, 0.1)
    
    # enabling other hues to be generated
    def randomnize_given_color(self, color):
        new_color = deepcopy(color)
        if r.random() < self.randomness:
            new_hue = np.mod(color[0] + r.uniform(30, 100), 360)
            color[0] = new_hue
            new_saturation = self.saturation_clip(color[1] + r.uniform(0, 0.2))
            color[1] = new_saturation
        return color
    
    def saturation_clip(self, value):
        value = abs(value)
        direction = (-1)**int(value)
        return np.mod((1 + direction*np.mod(value, 1)), 1)
    
    def stepping_wheel(self, color):
        self.num_steps= round(r.uniform(6, 8))
        rgb = self.color_library.hsl_to_rgb(color[0], color[1], color[2])
        # randomnize colors based on self.randomness to get more diverse outputs
        color = self.randomnize_given_color(color)
        rgb = self.color_library.hsl_to_rgb(color[0], color[1], color[2])
        print("given color: ", color)
        self.color_library.print_combo(rgb, rgb)
        
        # center_i: middle of the palette
        center_i = round(self.num_steps/2 + 0.1)-1
            
        # hue: linear increase
        palette_h = np.zeros(self.num_steps)
        hue_shift = self.generate_hue_shift()
        for i in range(0, self.num_steps):
            value = color[0] + (i - center_i) * hue_shift
            palette_h[i] = np.mod(value, 360)
        
        # saturation: proportional decrease
        palette_s = np.zeros(self.num_steps)         
        saturation_shift = 0.2
        palette_s[center_i] = np.clip(color[1], 0, 1)
        sign = 1
        for i in range(center_i - 1, -1, -1): # left
            last = palette_s[i + 1] # saturation of last element
            prop = 1.0/(1 + np.exp(-abs(i - center_i)/5.0))  # sigmoid
            value = sign*last - saturation_shift * prop
            sign = value / abs(value)
            palette_s[i] = self.saturation_clip(value)
        sign = 1
        for i in range(center_i + 1, self.num_steps, 1): # right   
            last = palette_s[i - 1] # saturation of last element
            prop = 1.0/(1 + np.exp(-abs(i - center_i)/5.0)) 
            value = sign*last - saturation_shift * prop
            sign = value / abs(value)
            palette_s[i] = self.saturation_clip(value)
        palette_s = np.clip(palette_s, 0.01, 0.99)
        
        # brightness: a*log(x)+1, a in [0.3, 1] (increasing)
        palette_b = np.zeros(self.num_steps)
        # determines the placement of the sampled color in the palette
        center_i = round(self.num_steps/(1.0 + np.exp(3-5*color[2]))) # modified sigmoid
        center_i = np.clip(int(center_i), 0, self.num_steps-1)
        print("center_i: ", center_i)
        palette_b[center_i] = np.clip(color[2], 0.01, 0.95) # given middle color
        delta_x = 0.05
        # setting a in the a*log(x) + 1
        if (color[2] > 0.49 and color[2] < 0.51): # heuristics for setting the rate of change of brightness
            a = 1.585
        else:
            a = abs(0.1/(color[2]-0.5))**0.2 
        print("a: ", a)
        for i in range(center_i - 1, -1, -1): # left
            last = palette_b[i + 1] # brightness of last element
            palette_b[i] = np.clip((last - delta_x * (a/last)**0.5), 0.01, 0.95) # derivative of a*log(x)+1  
        for i in range(center_i + 1, self.num_steps, 1): # right   
            last = palette_b[i - 1] # brightness of last element
            palette_b[i] = np.clip((last + delta_x * (a/last)**0.5), 0.01, 0.95) # derivative of a*log(x)+1
        palette_b = np.clip(palette_b, 0.01, 0.95)
        
        # combining h, s, b
        palette = []
        for i in range(0, self.num_steps):
            c = [palette_h[i], palette_s[i], palette_b[i]]
            palette.append(c) 
            rgb = self.color_library.hsl_to_rgb(c[0], c[1], c[2])
            self.color_library.print_combo(rgb, rgb)
            print(c)

        return palette
    
    
    # Given a list of inputs [colors] that are in hsl form [[h,s,l], [h,s,l], [h,s,l] ...]
    # Outputs colors in HSL: [[h,s,l], [h,s,l], [h,s,l]...]
    def sample_gmm(self, hsl_input, num_samples):
        hsl_input = np.reshape(hsl_input, (-1, 3)) # 3 columns
        self.color_gmm.fit(hsl_input)
        return self.color_gmm.sample(num_samples)[0]
    
    
    # takes in liked colors from quiz and liked palettes?
    # return palettes in form [[[h,s,l], [h,s,l]...], [[h,s,l], [h,s,l] ...]].
    def generate_palettes(self, samples, palettes, num_palettes, if_print):
        hsl_input = self.process_input(samples, palettes, if_print)
        gmm_samples = self.sample_gmm(hsl_input, num_palettes)
        palettes = []
        for color in gmm_samples: # make a palette for each sample from the gmm
            p = self.stepping_wheel(color)
            palettes.append(p)
            rgb1 = self.color_library.hsl_to_rgb(p[0][0], p[0][1], p[0][2])
            rgb2 = self.color_library.hsl_to_rgb(p[self.num_steps - 1][0], p[self.num_steps - 1][1], p[self.num_steps - 1][2])
            self.color_library.print_combo(rgb1, rgb2)
            self.color_library.print_combo(rgb2, rgb1)
            print("")
        return palettes
    
    # helper function that translates generated palettes into a json file. (outputs of generate_palettes)
    # inputs: palettes in form [ [ color, .., color], [color, ..., color] ]  where each color is hsl
    # output: a json file in format:
    # [{id: color1{label, hex, rgb}, {color2:label, hex, rgb}]
    def output_to_json(self, palettes):
        outputs = {}
        
        with io.open('our_fonts.txt', encoding='latin-1') as myfile:
        for i in myfile.readlines():
            font_list.append(i.rstrip() )
        
        for palette in palettes:
            c_counter = 1
            color_id = []
            p = {}
            for color in palette:
                c = {}
                rgb = self.color_library.hsl_to_rgb(color[0], color[1], color[2])
                c["rgb"] = self.color_library.arr_to_int(rgb)
                c["hex"] = self.color_library.rgb_to_hex(rgb[0], rgb[1], rgb[2])
                c["label"] = [self.color_library.color_descriptor(color[0], color[1], color[2])]
                color_string = "color" + str(c_counter)
                p[color_string] = c
                c_counter +=1
                color_id.append(c["hex"])
            
            outputs[hash(tuple(color_id))] = p # note: we want to order the palettes from light to dark before hashing!
        
        return outputs

small_pastel = [ [255, 228, 171], [255, 171, 209], [144, 240, 155], [245, 118, 130], [250, 178, 162], [145, 255, 187], [203, 240, 168]]
small_earth = [ [192, 87, 70], [240, 207, 101], [73, 67, 49], [89, 152, 197], [222, 185, 134], [208, 205, 148], [247, 208, 138]]
small_wack = [ [246,71,64], [248,221,164], [191, 219, 247], [60, 187, 177], [87, 226, 229], [241, 113, 5], [106, 16, 242]]


autumn = [ [225, 104, 101] ,[226, 114, 91] ,[222, 99, 96] ,[208, 95, 4] ,[167, 37, 37] ,[179, 45, 41] ,[180, 51, 50] ,[141, 63, 63] ,[255, 231, 114] ,[255, 239, 161] ,[223, 190, 111] ,[225, 188, 100] ,[96, 73, 19] ,[195, 153, 83] ,[202, 187, 72] ,[207, 181, 59] ,[185, 141, 40] ,[186, 127, 3] ,[220, 178, 12] ,[58, 106, 71] ,[57, 100, 19] ,[95, 167, 119] ,[88, 113, 86] ,[93, 94, 55] ,[168, 152, 155] ,[168, 101, 21] ,[171, 145, 122] ,[173, 120, 27] ,[169, 164, 145] ,[175, 89, 62] ,[136, 83, 66] ,[134, 148, 159] ,[138, 51, 36] ,[135, 124, 123] ,[245, 237, 239] ,[255, 228, 205] ,[77, 61, 20] ]
neon = [ [255, 73, 108] ,[255, 83, 73] ,[255, 43, 43] ,[218, 38, 71] ,[255, 64, 64] ,[205, 74, 76] ,[255, 36, 0] ,[254, 254, 34] ,[253, 255, 0] ,[255, 244, 79] ,[255, 219, 0] ,[69, 206, 162] ,[0, 166, 147] ,[197, 227, 132] ,[0, 127, 255] ,[0, 255, 255] ,[29, 172, 214] ,[31, 194, 194] ,[156, 81, 182] ,[223, 115, 255] ,[219, 145, 239] ,[139, 0, 255] ]
underwater = [ [244, 196, 48] ,[240, 213, 45] ,[236, 242, 69] ,[225, 104, 101] ,[222, 99, 96] ,[226, 114, 91] ,[27, 101, 157] ,[25, 89, 168] ,[20, 80, 170] ,[15, 45, 158] ,[8, 232, 222] ,[24, 167, 181] ,[3, 106, 110] ,[0, 204, 204] ,[0, 255, 255] ,[28, 169, 201] ,[0, 51, 153] ,[0, 86, 167] ,[0, 35, 135] ,[31, 117, 254] ,[25, 116, 210] ,[0, 47, 167] ,[37, 150, 209] ,[32, 46, 84] ,[37, 31, 79] ,[65, 105, 225] ,[59, 145, 180] ]

medium_contrast=[
[[194, 231, 217],[38, 63, 139]],
[[252, 109, 171],[247, 246, 197]],
[[152, 210, 235],[178, 177, 207]],
[[255, 192, 159],[252, 245, 199]],
[[281, 224, 242],[82, 21, 78]],
[[18, 69, 89],[174, 195, 176]]
]

generator = palette_generator()
palettes = generator.generate_palettes(underwater, medium_contrast, 10, True)

given samples:
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
---
given palettes:
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
---
given color:  [313.30425939   0.71124059   0.37141739]
Lorem ipsum.
center_i:  1
a:  0.9509628555274782
Lorem ipsum.
[307.90761906708434, 0.48153626212454403, 0.2914117407531311]
Lorem ipsum.
[310.605939230708, 0.6012737941470343, 0.37141739173777316]
Lorem ipsum.
[313.3042593943316, 0.71124059360953, 0.45142304272241524]
Lorem ipsum.
[316.00257955795524, 0.6012737941470343, 0.5239935099807889]
Lorem ipsum.
[318.7008997215789, 0.48153626212454403, 0.

given color:  [197.68129053   0.8106619    0.23730023]
Lorem ipsum.
center_i:  1
a:  0.8243432023173519
Lorem ipsum.
[194.33009864851232, 0.4518263084029046, 0.14410901730856052]
Lorem ipsum.
[195.44716260920987, 0.5809575696480636, 0.23730022857446628]
Lorem ipsum.
[196.5642265699074, 0.7006951016705538, 0.33049143984037205]
Lorem ipsum.
[197.68129053060494, 0.8106619011330494, 0.4094581285546107]
Lorem ipsum.
[198.7983544913025, 0.7006951016705538, 0.4804027190578944]
Lorem ipsum.
[199.915418452, 0.5809575696480636, 0.5458997037122431]
Lorem ipsum.
[201.03248241269756, 0.4518263084029046, 0.6073420236690573]
Lorem ipsum.
Lorem ipsum.

given color:  [216.47260409   0.97159166   0.72276683]
Lorem ipsum.
center_i:  5
a:  0.8519809727285529
Lorem ipsum.
[202.615795484866, 0.612756062548848, 0.4251178226563218]
Lorem ipsum.
[207.23473168760458, 0.7418873237940069, 0.4909824114719883]
Lorem ipsum.
[211.85366789034316, 0.8616248558164974, 0.5530415945223007]
Lorem ipsum.
[216.4726040930817,

In [23]:
# Overall: hue difference/themes/light vs. dark -- Katherine
autumn = [ [225, 104, 101] ,[226, 114, 91] ,[222, 99, 96] ,[208, 95, 4] ,[167, 37, 37] ,[179, 45, 41] ,[180, 51, 50] ,[141, 63, 63] ,[255, 231, 114] ,[255, 239, 161] ,[223, 190, 111] ,[225, 188, 100] ,[96, 73, 19] ,[195, 153, 83] ,[202, 187, 72] ,[207, 181, 59] ,[185, 141, 40] ,[186, 127, 3] ,[220, 178, 12] ,[58, 106, 71] ,[57, 100, 19] ,[95, 167, 119] ,[88, 113, 86] ,[93, 94, 55] ,[168, 152, 155] ,[168, 101, 21] ,[171, 145, 122] ,[173, 120, 27] ,[169, 164, 145] ,[175, 89, 62] ,[136, 83, 66] ,[134, 148, 159] ,[138, 51, 36] ,[135, 124, 123] ,[245, 237, 239] ,[255, 228, 205] ,[77, 61, 20] ]
underwater = [ [244, 196, 48] ,[240, 213, 45] ,[236, 242, 69] ,[225, 104, 101] ,[222, 99, 96] ,[226, 114, 91] ,[27, 101, 157] ,[25, 89, 168] ,[20, 80, 170] ,[15, 45, 158] ,[8, 232, 222] ,[24, 167, 181] ,[3, 106, 110] ,[0, 204, 204] ,[0, 255, 255] ,[28, 169, 201] ,[0, 51, 153] ,[0, 86, 167] ,[0, 35, 135] ,[31, 117, 254] ,[25, 116, 210] ,[0, 47, 167] ,[37, 150, 209] ,[32, 46, 84] ,[37, 31, 79] ,[65, 105, 225] ,[59, 145, 180] ]

monochrome =  [[ [250, 224, 181] ,[224, 159, 62] ] , [[158, 42, 43] ,[84, 11, 14] ], [[65, 54, 32] ,[159, 120, 51]]]

very_different = [
[[154, 210, 203],[254, 255, 190]],
[[173, 245, 255], [71 , 40, 54]],
[[233, 114, 76], [189, 198, 150]],
[ [20, 49, 9], [245, 233, 226]],
[[137, 189, 158],[240, 201, 135]]]

mixed_palettes=[
[[194, 231, 217],[38, 63, 139]],
[[252, 109, 171],[247, 246, 197]],
[[152, 210, 235],[178, 177, 207]],
[[255, 192, 159],[252, 245, 199]],
[[281, 224, 242],[82, 21, 78]],
[[18, 69, 89],[174, 195, 176]]
]


generator = palette_generator()

# autumn inputs, with different kinds of liked palettes
palettes = generator.generate_palettes(autumn, monochrome, 10, True)
palettes = generator.generate_palettes(autumn, mixed_palettes, 10, True)
palettes = generator.generate_palettes(autumn, very_different, 10, True)

# underwater themed inputs, with different kinds of liked palettes
# palettes = generator.generate_palettes(underwater, monochrome, 10, True)
# palettes = generator.generate_palettes(underwater, mixed_palettes, 10, True)
# palettes = generator.generate_palettes(underwater, very_different, 10, True)



given samples:
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
---
given palettes:
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
Lorem ipsum.
---
given color:  [-17.21488814   0.42210452   0.59505337]
Lorem ipsum.
center_i:  4
a:  1.0101979829315213
Lorem ipsum.
[342.31486543366884, 0.0632689224047458, 0.3060458123241695]
Lorem ipsum.
[342.471614241931, 0.19240018364990497, 0.38684462338814773]
Lorem ipsum.
[342.6283630501931, 0.31213771567239545, 0.46087053767619574]
Lorem ipsum.
[342.7851118584553, 0.422104515134891, 0.5299062382962546]
Lor

center_i:  3
a:  1.5191400811506455
Lorem ipsum.
[129.38979923143455, 0.02671069762189915, 0.18970990073867167]
Lorem ipsum.
[131.0933700684671, 0.10242056362326002, 0.301874476495685]
Lorem ipsum.
[132.79694090549964, 0.2221580956457505, 0.3993893127449625]
Lorem ipsum.
[134.50051174253218, 0.33212489510824605, 0.48764021579706235]
Lorem ipsum.
[136.20408257956473, 0.2221580956457505, 0.5758911188491622]
Lorem ipsum.
[137.90765341659727, 0.10242056362326002, 0.6570991020596723]
Lorem ipsum.
[139.6112242536298, 0.02671069762189915, 0.7331235856240872]
Lorem ipsum.
[141.31479509066236, 0.16470559384742156, 0.8050983612554242]
Lorem ipsum.
Lorem ipsum.

given color:  [171.2534452    0.45042614   0.61622155]
Lorem ipsum.
center_i:  4
a:  0.9703818521881771
Lorem ipsum.
[144.02471074458356, 0.09159054901393704, 0.34041316221372264]
Lorem ipsum.
[153.10095556378172, 0.2207218102590962, 0.4167128709258857]
Lorem ipsum.
[162.17720038297992, 0.3404593422815867, 0.4872723632268049]
Lorem ipsum.

[90.89148141183752, 0.3389936485017939, 0.862463053713834]
Lorem ipsum.
Lorem ipsum.

given color:  [29.30071686  0.53210326  0.37581691]
Lorem ipsum.
center_i:  2
a:  0.9576074375452961
Lorem ipsum.
[333.0397092531506, 0.1732676667857176, 0.20607133685766021]
Lorem ipsum.
[351.7933784557647, 0.30239892803087676, 0.2960035533557368]
Lorem ipsum.
[10.547047658378819, 0.422136460053367, 0.37581691413917706]
Lorem ipsum.
[29.30071686099295, 0.5321032595158626, 0.4556302749226173]
Lorem ipsum.
[48.05438606360708, 0.422136460053367, 0.5281168320655499]
Lorem ipsum.
[66.8080552662212, 0.30239892803087676, 0.5954453035146625]
Lorem ipsum.
[85.56172446883534, 0.1732676667857176, 0.6588531235674047]
Lorem ipsum.
Lorem ipsum.

given color:  [35.13949716  0.39822627  0.52316931]
Lorem ipsum.
center_i:  3
a:  1.3397303476529758
Lorem ipsum.
[347.79831083312286, 0.039390675187620516, 0.2592548844421279]
Lorem ipsum.
[3.5787062750426344, 0.16852193643277968, 0.356220792559314]
Lorem ipsum.
[19.35910